# Load Data and Merge into Proper DF to Save as CSV file

In [1]:
import os
os.chdir('../saliency_model/')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib

import pandas as pd
import seaborn as sns
from glob import glob

matplotlib.rcParams.update({'font.size': 15})

### Pick Metric
NSS, SIM, IG, AUC

In [2]:
plot_metric = 'IG'

In [3]:
# pick directory
save_dir ='../report/plots/'

In [4]:
os.chdir('../results/')
glob('*.p')

['19-02-01_results_over_baseline_faces_07.p',
 '19-02-01_results_plain_models.p',
 '19-02-01_results_face_07.p',
 '2019-05-12_results-with-std.p',
 '19-02-01_results_over_baseline_faces_05.p',
 '2019-05-11_results-with-std.p',
 '19-02-01_results_over_baseline_faces_03.p',
 '19-01-31_results.p',
 '19-02-01_results_face_03.p',
 '19-02-01_results_face_05.p']

## Pick the Dataset and Load Required Data

In [5]:
(summary, context, model, metric) = pickle.load(open( '19-02-01_results_plain_models.p', "rb"))

In [6]:
print(np.shape(summary))
print(context)
print(model)
print(metric)

(6, 9, 4)
['all', 'person', 'no person', 'animal', 'electronic', 'sports']
['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg', 'faces']
['nss', 'sim', 'ig', 'auc']


In [7]:
(summary_f03, context_f03, model_f03, metric_f03) = pickle.load(open( '19-02-01_results_face_03.p', "rb"))
(summary_f05, context_f05, model_f05, metric_f05) = pickle.load(open( '19-02-01_results_face_05.p', "rb"))
(summary_f07, context_f07, model_f07, metric_f07) = pickle.load(open( '19-02-01_results_face_07.p', "rb"))

#### Vizualize the inside of one of the models

In [8]:
model_f03

['aim_face_03',
 'cas_face_03',
 'covsal_face_03',
 'gbvs_face_03',
 'icf_face_03',
 'ik_face_03',
 'sun_face_03']

## Data Loading

### Get ID of the Metric

In [9]:
if plot_metric == 'NSS':
    metric_id = 0
elif plot_metric == 'SIM':
    metric_id = 1
elif plot_metric == 'IG':
    metric_id = 2
elif plot_metric == 'AUC':
    metric_id = 3
else:
    raise ValueError('Wrong metric name provided.')
    
metric_id

1

### Load the scores

In [10]:
scores = summary[:,:,metric_id]
f03_scores = summary_f03[:,:,metric_id]
f05_scores = summary_f05[:,:,metric_id]
f07_scores = summary_f07[:,:,metric_id]

## Change scores into DataFrames
### No Faces Data Frame

In [11]:
df = pd.DataFrame(data=scores, columns=model, index=context)
df.columns.name='models'
df.index.name='context'

df = df.transpose()
df

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.529256,0.504981,0.568607,0.472232,0.535764,0.478441
cas,0.521473,0.496486,0.561978,0.459220,0.525020,0.490274
covsal,0.528376,0.520976,0.540371,0.516241,0.507251,0.502744
gbvs,0.579692,0.547139,0.632463,0.532250,0.586402,0.524310
icf,0.540450,0.520813,0.572283,0.480500,0.534348,0.493894
ik,0.483443,0.458229,0.524316,0.440349,0.498116,0.439303
sun,0.436436,0.400730,0.494317,0.374971,0.449312,0.347440
dg,0.642767,0.630332,0.662924,0.623571,0.637034,0.623714
faces,0.380547,0.381124,0.324046,0.281475,0.410412,0.323161


#### Separate Faces From the rest of the results

In [12]:
faces = df.loc['faces']
df = df.loc[['aim','cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg']]

#### Get Mean Values across Contexts and Models

In [13]:
def calculate_mean_context_and_model(df):
    df['mean context'] = df.mean(axis=1)
    df = df.transpose()
    df['mean model'] = df.mean(axis=1)
    return df.transpose()     

In [14]:
df = calculate_mean_context_and_model(df)
df

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.529256,0.504981,0.568607,0.472232,0.535764,0.478441,0.514880
cas,0.521473,0.496486,0.561978,0.459220,0.525020,0.490274,0.509075
covsal,0.528376,0.520976,0.540371,0.516241,0.507251,0.502744,0.519326
gbvs,0.579692,0.547139,0.632463,0.532250,0.586402,0.524310,0.567043
icf,0.540450,0.520813,0.572283,0.480500,0.534348,0.493894,0.523715
ik,0.483443,0.458229,0.524316,0.440349,0.498116,0.439303,0.473959
sun,0.436436,0.400730,0.494317,0.374971,0.449312,0.347440,0.417201
dg,0.642767,0.630332,0.662924,0.623571,0.637034,0.623714,0.636724
mean model,0.532737,0.509961,0.569657,0.487417,0.534156,0.487515,0.520240


In [15]:
df_mean_model = df.loc['mean model']
df_mean_context = df.transpose().loc['mean context']

### Faces with $\alpha$ = 0.3

In [16]:
df_f03 = pd.DataFrame(data=f03_scores, columns=model_f03, index=context_f03)
df_f03.columns.name='models'
df_f03.index.name='context'

# Rename columns of the face plot
col = {'aim_face_03':'aim', 'cas_face_03':'cas', 'covsal_face_03':'covsal', 'gbvs_face_03':'gbvs',
       'icf_face_03':'icf', 'ik_10_face_03':'ik_10', 'sun_face_03':'sun', 'ik_face_03':'ik'}
df_f03 = df_f03.rename(index=str, columns=col).transpose()

df_f03

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.535489,0.515820,0.567375,0.473084,0.547418,0.482798
cas,0.525728,0.503291,0.562292,0.459744,0.533971,0.493499
covsal,0.540867,0.540492,0.541474,0.519671,0.528012,0.510934
gbvs,0.583638,0.553899,0.631847,0.532606,0.594117,0.527445
icf,0.547291,0.531809,0.572388,0.482032,0.545444,0.500319
ik,0.490019,0.468970,0.524140,0.441561,0.508954,0.445144
sun,0.441578,0.409820,0.493058,0.375098,0.458578,0.350194


#### Get Mean Values across Contexts and Models

In [17]:
df_f03 = calculate_mean_context_and_model(df_f03)
df_f03

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.535489,0.515820,0.567375,0.473084,0.547418,0.482798,0.520331
cas,0.525728,0.503291,0.562292,0.459744,0.533971,0.493499,0.513087
covsal,0.540867,0.540492,0.541474,0.519671,0.528012,0.510934,0.530241
gbvs,0.583638,0.553899,0.631847,0.532606,0.594117,0.527445,0.570592
icf,0.547291,0.531809,0.572388,0.482032,0.545444,0.500319,0.529880
ik,0.490019,0.468970,0.524140,0.441561,0.508954,0.445144,0.479798
sun,0.441578,0.409820,0.493058,0.375098,0.458578,0.350194,0.421388
mean model,0.523516,0.503443,0.556082,0.469114,0.530928,0.472905,0.509331


In [18]:
df_f03_mean_model = df_f03.loc['mean model']
df_f03_mean_context = df_f03.transpose().loc['mean context']

### Faces with $\alpha$ = 0.5

In [19]:
df_f05 = pd.DataFrame(data=f05_scores, columns=model_f05, index=context_f05)
df_f05.columns.name='models'
df_f05.index.name='context'

# Rename columns of the face plot
col = {'aim_face_05':'aim', 'cas_face_05':'cas', 'covsal_face_05':'covsal', 'gbvs_face_05':'gbvs',
       'icf_face_05':'icf', 'ik_10_face_05':'ik_10', 'sun_face_05':'sun', 'ik_face_05':'ik'}
df_f05 = df_f05.rename(index=str, columns=col).transpose()

df_f05

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.542495,0.527066,0.567506,0.475102,0.558768,0.488953
cas,0.530474,0.511503,0.561227,0.461105,0.540870,0.497408
covsal,0.550315,0.555510,0.541893,0.522584,0.543866,0.518601
gbvs,0.588542,0.561783,0.631919,0.533836,0.602438,0.531272
icf,0.552661,0.540406,0.572526,0.483636,0.554154,0.504156
ik,0.496925,0.480040,0.524296,0.443189,0.519886,0.451064
sun,0.448339,0.420649,0.493226,0.376733,0.469726,0.355033


#### Get Mean Values across Contexts and Models

In [20]:
df_f05 = calculate_mean_context_and_model(df_f05)
df_f05

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.542495,0.527066,0.567506,0.475102,0.558768,0.488953,0.526648
cas,0.530474,0.511503,0.561227,0.461105,0.540870,0.497408,0.517098
covsal,0.550315,0.555510,0.541893,0.522584,0.543866,0.518601,0.538795
gbvs,0.588542,0.561783,0.631919,0.533836,0.602438,0.531272,0.574965
icf,0.552661,0.540406,0.572526,0.483636,0.554154,0.504156,0.534590
ik,0.496925,0.480040,0.524296,0.443189,0.519886,0.451064,0.485900
sun,0.448339,0.420649,0.493226,0.376733,0.469726,0.355033,0.427284
mean model,0.529964,0.513851,0.556085,0.470884,0.541387,0.478070,0.515040


In [21]:
df_f05_mean_model = df_f05.loc['mean model']
df_f05_mean_context = df_f05.transpose().loc['mean context']

### Faces with $\alpha$ = 0.7

In [22]:
df_f07 = pd.DataFrame(data=f07_scores, columns=model_f07, index=context_f07)
df_f07.columns.name='models'
df_f07.index.name='context'

# Rename columns of the face plot
col = {'aim_face_07':'aim', 'cas_face_07':'cas', 'covsal_face_07':'covsal', 'gbvs_face_07':'gbvs',
       'icf_face_07':'icf', 'ik_10_face_07':'ik_10', 'sun_face_07':'sun', 'ik_face_07':'ik'}
df_f07 = df_f07.rename(index=str, columns=col).transpose()

df_f07

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.549862,0.539012,0.567449,0.477050,0.569102,0.496495
cas,0.538440,0.524250,0.561443,0.463732,0.553208,0.503983
covsal,0.555271,0.563686,0.541629,0.524548,0.551776,0.524843
gbvs,0.595868,0.573428,0.632245,0.536224,0.614022,0.537579
icf,0.557546,0.548177,0.572734,0.485544,0.562260,0.506198
ik,0.505718,0.494144,0.524479,0.445401,0.532979,0.459770
sun,0.458123,0.436317,0.493472,0.379859,0.485290,0.363897


In [23]:
df_f07 = calculate_mean_context_and_model(df_f07)
df_f07

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.549862,0.539012,0.567449,0.477050,0.569102,0.496495,0.533162
cas,0.538440,0.524250,0.561443,0.463732,0.553208,0.503983,0.524176
covsal,0.555271,0.563686,0.541629,0.524548,0.551776,0.524843,0.543626
gbvs,0.595868,0.573428,0.632245,0.536224,0.614022,0.537579,0.581561
icf,0.557546,0.548177,0.572734,0.485544,0.562260,0.506198,0.538743
ik,0.505718,0.494144,0.524479,0.445401,0.532979,0.459770,0.493749
sun,0.458123,0.436317,0.493472,0.379859,0.485290,0.363897,0.436159
mean model,0.537261,0.525574,0.556207,0.473194,0.552662,0.484681,0.521597


In [24]:
df_f07_mean_model = df_f07.loc['mean model']
df_f07_mean_context = df_f07.transpose().loc['mean context']

### Create Mean Matrices

In [25]:
mean_models = pd.DataFrame([df_mean_model, df_f03_mean_model, df_f05_mean_model, df_f07_mean_model], index=[0,0.3,0.5,0.7])

mean_models = mean_models.transpose()
mean_models = mean_models.loc[['all', 'person', 'no person', 'animal', 'electronic', 'sports']]
mean_models

,0.0,0.3,0.5,0.7
context,,,,
all,0.532737,0.523516,0.529964,0.537261
person,0.509961,0.503443,0.513851,0.525574
no person,0.569657,0.556082,0.556085,0.556207
animal,0.487417,0.469114,0.470884,0.473194
electronic,0.534156,0.530928,0.541387,0.552662
sports,0.487515,0.472905,0.478070,0.484681


In [26]:
mean_contexts = pd.DataFrame([df_mean_context, df_f03_mean_context, df_f05_mean_context, df_f07_mean_context], index=[0,0.3,0.5,0.7])

mean_contexts = mean_contexts.transpose()
mean_contexts = mean_contexts.loc[['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun']]
mean_contexts

,0.0,0.3,0.5,0.7
aim,0.514880,0.520331,0.526648,0.533162
cas,0.509075,0.513087,0.517098,0.524176
covsal,0.519326,0.530241,0.538795,0.543626
gbvs,0.567043,0.570592,0.574965,0.581561
icf,0.523715,0.529880,0.534590,0.538743
ik,0.473959,0.479798,0.485900,0.493749
sun,0.417201,0.421388,0.427284,0.436159


## Diff of Context 

#### Drop DG for diff

In [27]:
df1 = df.loc[['aim','cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'mean model']]
df1

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.529256,0.504981,0.568607,0.472232,0.535764,0.478441,0.514880
cas,0.521473,0.496486,0.561978,0.459220,0.525020,0.490274,0.509075
covsal,0.528376,0.520976,0.540371,0.516241,0.507251,0.502744,0.519326
gbvs,0.579692,0.547139,0.632463,0.532250,0.586402,0.524310,0.567043
icf,0.540450,0.520813,0.572283,0.480500,0.534348,0.493894,0.523715
ik,0.483443,0.458229,0.524316,0.440349,0.498116,0.439303,0.473959
sun,0.436436,0.400730,0.494317,0.374971,0.449312,0.347440,0.417201
mean model,0.532737,0.509961,0.569657,0.487417,0.534156,0.487515,0.520240


### Substract to get the difference with Faces

In [28]:
df_diff_f03 = ((df_f03 - df1) / df1)*100
df_diff_f05 = ((df_f05 - df1) / df1)*100
df_diff_f07 = ((df_f07 - df1) / df1)*100

#### Recalculate Means

In [29]:
df_diff_f03 = calculate_mean_context_and_model(df_diff_f03)
df_diff_f05 = calculate_mean_context_and_model(df_diff_f05)
df_diff_f07 = calculate_mean_context_and_model(df_diff_f07)

In [30]:
df_diff_f03['mean context'] = df_diff_f03.mean(axis=1)
df_diff_f03 = df_diff_f03.transpose()
df_diff_f03['mean model'] = df_diff_f03.mean(axis=1)
df_diff_f03 = df_diff_f03.transpose()


In [31]:
df_diff_f03

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,1.177735,2.146296,-0.216663,0.180400,2.175090,0.910813,1.062203
cas,0.815888,1.370441,0.055934,0.114155,1.704735,0.657708,0.786510
covsal,2.364020,3.746011,0.204147,0.664419,4.092846,1.629007,2.116436
gbvs,0.680635,1.235511,-0.097500,0.066931,1.315662,0.597906,0.633042
icf,1.265682,2.111228,0.018284,0.318745,2.076516,1.300859,1.181791
ik,1.360209,2.344008,-0.033566,0.275193,2.175635,1.329484,1.241623
sun,1.178189,2.268435,-0.254555,0.033862,2.062375,0.792838,1.013321
mean model,1.216412,2.120613,-0.082787,0.173879,2.184708,0.968291,1.096721


#### Get mean of models and mean of contexts in diff out

#### Diff Models (analyse across contexts)

In [32]:
mean_model_diff_f03 = df_diff_f03.loc['mean model']
mean_model_diff_f05 = df_diff_f05.loc['mean model']
mean_model_diff_f07 = df_diff_f07.loc['mean model']

mean_models_diff = pd.DataFrame([mean_model_diff_f03, 
                            mean_model_diff_f05,
                            mean_model_diff_f07], index=[0.3,0.5,0.7])

mean_models_diff = mean_models_diff.transpose()
mean_models_diff

,0.3,0.5,0.7
context,,,
all,1.216412,2.145142,3.577095
person,2.120613,3.869814,6.295990
no person,-0.082787,-0.337045,-0.315328
animal,0.173879,0.115064,0.617690
electronic,2.184708,3.904918,6.096709
sports,0.968291,1.642154,3.100225
mean context,1.096721,1.888230,3.224735


In [33]:
mean_context_diff_f03 = df_diff_f03.transpose().loc['mean context']
mean_context_diff_f05 = df_diff_f05.transpose().loc['mean context']
mean_context_diff_f07 = df_diff_f07.transpose().loc['mean context']

mean_context_diff = pd.DataFrame([mean_context_diff_f03, 
                            mean_context_diff_f05,
                            mean_context_diff_f07], index=[0.3,0.5,0.7])

mean_context_diff = mean_context_diff.transpose()
mean_context_diff

,0.3,0.5,0.7
models,,,
aim,1.062203,2.295003,3.570811
cas,0.786510,1.582494,2.980620
covsal,2.116436,3.773192,4.711853
gbvs,0.633042,1.410697,2.586948
icf,1.181791,2.082437,2.875759
ik,1.241623,2.536625,4.208146
sun,1.013321,2.441900,4.610065
mean model,1.096721,1.888230,3.224735


## Vizualize relative errors

## Diff of Models

## List all the df and save them

In [34]:
df_feaces = pd.DataFrame(data=faces)

In [35]:
os.chdir('../notebooks/data/')

In [36]:
df.to_csv('scores_' + plot_metric + '.csv')
df_feaces.to_csv('scores_faces_' + plot_metric + '.csv')
df_f03.to_csv('scores_f03_' + plot_metric + '.csv')
df_f05.to_csv('scores_f05_' + plot_metric + '.csv')
df_f07.to_csv('scores_f07_' + plot_metric + '.csv')

mean_models.to_csv('scores_mean_models_' + plot_metric + '.csv')
mean_contexts.to_csv('scores_mean_contexts_' + plot_metric + '.csv')

mean_models_diff.to_csv('scores_mean_models_diff_' + plot_metric + '.csv')
mean_context_diff.to_csv('scores_mean_contexts_diff_' + plot_metric + '.csv')